This code doesn't create a cluster and run jobs on it. It just manipulates a PySpark Dataframe.

In [70]:
print("Hi Gany, This is a sample PySpark installation")

Hi Gany, This is a sample PySpark installation


In [71]:
#This doesn't work from the notebook, but works from the command line
!brew install apache-spark
#This doesn't help either from here or the command line. Download java through the browser.
# But looks like the brew install from command line actually worked.. look at the PATH and 
# JAVA_HOME variables in the below cell. They are brew based.
!brew install openjdk@11

Running `brew update --auto-update`...
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).

You have 6 outdated formulae installed.
You can upgrade them with brew upgrade
or list them with brew outdated.

To reinstall 3.3.0, run:
  brew reinstall apache-spark
To reinstall 11.0.16.1_1, run:
  brew reinstall openjdk@11


In [72]:
!export SPARK_HOME=/opt/homebrew/Cellar/apache-spark/3.3.0/libexec
!export PATH=/opt/homebrew/Cellar/apache-spark/3.3.0/bin:$PATH
!export PATH=/opt/homebrew/opt/openjdk@11/bin/java:$PATH
# looks the case of the JAVA_HOME variable made all the difference    
!export JAVA_HOME=/opt/homebrew/opt/openjdk@11/bin/java
!export PYSPARK_SUBMIT_ARGS="--master local[3] pyspark-shell"

In [73]:
!java -version

java version "1.8.0_351"
Java(TM) SE Runtime Environment (build 1.8.0_351-b10)
Java HotSpot(TM) 64-Bit Server VM (build 25.351-b10, mixed mode)


In [74]:
!pip3 install pyspark
!pip3 install findspark



[notice] A new release of pip available: 22.2 -> 22.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [75]:
import findspark
findspark.init()

In [76]:
# To create a mock dataframe
columns = ["language","users_count"]
data = [("Java", "20000"), ("Python", "100000"), ("Scala", "3000")]

In [77]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

df = spark.createDataFrame(data,columns)
df.show()

+--------+-----------+
|language|users_count|
+--------+-----------+
|    Java|      20000|
|  Python|     100000|
|   Scala|       3000|
+--------+-----------+



In [78]:
challenge_df = spark.read.format("csv").option("header", "true").load("/Users/gchidam/challenge.csv")

In [79]:
challenge_df.show()

+---------------+--------------+-----------------+----------+
|     ip_address|       Country|      Domain Name|Bytes_used|
+---------------+--------------+-----------------+----------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|
| 119.239.207.13|         China|         youtu.be|        51|
|  68.69.217.210|         China|        adobe.com|        10|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|
|   211.13.10.68|     Indonesia|          hud.gov|        29|
|   239.80.21.97|      Suriname|       smh.com.au|       218|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|
| 127.242.24.138|         China| surveymonkey.com|       123|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|
|   237.54.11.63|         China|       amazon.com|        83|
| 252.141.157.25|         Japan|      cornell.edu|       374|
|185.220.128.248|       Belgium|       weebly.com|       389|
|   151.77.19.45|   Afghanistan|independent.co.uk|       282|
|  9.161

In [80]:
# This import is needed to run the when method etc. 
from pyspark.sql.functions import *
challenge_df = challenge_df.withColumn("Mexico", when(challenge_df.Country == "Mexico", "Yes").otherwise("No"))
challenge_df.show()

+---------------+--------------+-----------------+----------+------+
|     ip_address|       Country|      Domain Name|Bytes_used|Mexico|
+---------------+--------------+-----------------+----------+------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|    No|
| 119.239.207.13|         China|         youtu.be|        51|    No|
|  68.69.217.210|         China|        adobe.com|        10|    No|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|    No|
|   211.13.10.68|     Indonesia|          hud.gov|        29|    No|
|   239.80.21.97|      Suriname|       smh.com.au|       218|    No|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|    No|
| 127.242.24.138|         China| surveymonkey.com|       123|    No|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|    No|
|   237.54.11.63|         China|       amazon.com|        83|    No|
| 252.141.157.25|         Japan|      cornell.edu|       374|    No|
|185.220.128.248|       Belgium|  

In [81]:
bytes_agg_df = challenge_df.groupBy("Mexico").agg(sum("Bytes_used"))
bytes_agg_df.show()

+------+---------------+
|Mexico|sum(Bytes_used)|
+------+---------------+
|    No|       508076.0|
|   Yes|         6293.0|
+------+---------------+



In [84]:
unique_ip_count_df = challenge_df.groupBy("Country").agg(countDistinct("ip_address"))
unique_ip_count_df.show()

+-----------+-----------------+
|    Country|count(ip_address)|
+-----------+-----------------+
|       Chad|                1|
|     Russia|               56|
|   Paraguay|                1|
|      Yemen|                1|
|     Sweden|               28|
|Philippines|               65|
|   Malaysia|                5|
|     Turkey|                1|
|     Malawi|                2|
|    Germany|                5|
|    Comoros|                1|
|Afghanistan|                5|
|     Rwanda|                1|
|      Sudan|                1|
|     France|               21|
|     Greece|                8|
|  Sri Lanka|                3|
|   Dominica|                1|
|  Argentina|               14|
|    Belgium|                1|
+-----------+-----------------+
only showing top 20 rows

